# TechStockData_monthly (fixed)Run all cells top‑to‑bottom. Requires only FRED key; Polygon/Finnhub sections are optional.

In [ ]:
from dotenv import load_dotenv, find_dotenvload_dotenv(find_dotenv())print('dotenv loaded')

In [ ]:
# Tech Monthly (fixed)# - Robust .env loading (find_dotenv)# - Works with only FRED key (Polygon/Finnhub optional; sections skip gracefully)# - Auto-downloads NLTK VADER if missing# - Uses 'ME' month-end resampling (avoids FutureWarning)# - Saves CSVs to ../data_cache/Monthly# - Includes simple OLS with safety checksimport os, time, json, datetime as dtfrom typing import List, Dict, Anyimport requestsimport pandas as pdimport numpy as nptry:    import yfinance as yfexcept Exception as _e:    raise SystemExit("Please install yfinance: pip install yfinance") from _eimport matplotlib.pyplot as plt# Optional: statsmodels for OLS (script will still run without it, just skips OLS)try:    import statsmodels.api as sm    _STATSMODELS_OK = Trueexcept Exception:    _STATSMODELS_OK = False# dotenv robust loadtry:    from dotenv import load_dotenv, find_dotenv    load_dotenv(find_dotenv())  # finds ../.env tooexcept Exception:    pass# NLTK VADER auto-setup (optional)_VADER_OK = Falsetry:    import nltk    try:        from nltk.sentiment import SentimentIntensityAnalyzer  # noqa: F401    except LookupError:        nltk.download('vader_lexicon')    from nltk.sentiment import SentimentIntensityAnalyzer    _VADER_OK = Trueexcept Exception:    _VADER_OK = False# ------------------ Keys & controls ------------------FRED_KEY     = os.getenv("af545f1dee482a6552309d2dd2e06fda")POLYGON_KEY  = os.getenv("61CPwAgcKao5Mm0N79i_jw3iZ5oAibST")FINNHUB_KEY  = os.getenv("d3i7slpr01qr304gpbr0d3i7slpr01qr304gpbrg")FRED_BASE    = "https://api.stlouisfed.org/fred/series/observations"POLY_BASE    = "https://api.polygon.io"FINNHUB_BASE = "https://finnhub.io/api/v1"OUT_DIR = "../data_cache/Monthly"os.makedirs(OUT_DIR, exist_ok=True)TECH_TICKERS = ["AAPL","MSFT","GOOGL","NVDA","META","AMZN"]AI_BASKET    = ["NVDA","META","MSFT","GOOGL","AMD","AVGO"]START  = "2018-01-01"END    = dt.date.today().isoformat()LAGS   = [1,3,6]print("Keys present:", {"FRED": bool(FRED_KEY), "POLYGON": bool(POLYGON_KEY), "FINNHUB": bool(FINNHUB_KEY)})print("Universe:", TECH_TICKERS)# ------------------ Helpers ------------------def _get_json(url: str, params: Dict[str, Any], max_retries=3, backoff=1.0):    last_err = None    for i in range(max_retries):        try:            r = requests.get(url, params=params, timeout=30)            if r.status_code == 429:                time.sleep(backoff * (i+1) * 2)                continue            r.raise_for_status()            try:                return r.json()            except json.JSONDecodeError as e:                last_err = e                time.sleep(backoff)                continue        except Exception as e:            last_err = e            time.sleep(backoff)    raise RuntimeError(f"GET failed after retries: {url} | last_err={last_err}")def _get_json_soft(url: str, params: Dict[str, Any], max_retries=3, backoff=1.5):    try:        return _get_json(url, params, max_retries=max_retries, backoff=backoff)    except Exception as e:        print(f"[warn] _get_json_soft failed: {url} | {type(e).__name__}: {e}")        return Nonedef _to_csv(df: pd.DataFrame, path: str):    df.to_csv(path, index=True)    print(f"Saved → {path}")def make_lags(df: pd.DataFrame, cols: List[str], lags=(1,3,6)) -> pd.DataFrame:    out = df.copy()    for c in cols:        if c not in out.columns:            continue        for L in lags:            out[f"{c}_lag{L}"] = out[c].shift(L)    return outdef diagnostics(df: pd.DataFrame, name: str, top=10):    print(f"\n[Diag] {name}: shape={df.shape}, index=({df.index.min()}, {df.index.max()})")    if df.index.duplicated().any():        dup = df.index[df.index.duplicated()]        print(f"[Diag] WARNING: duplicate index rows={len(dup)}")    na_pct = df.isna().mean().sort_values(ascending=False).head(top)    print("[Diag] Top columns by NaN%:\n", na_pct.to_string())# ------------------ FRED macro (monthly) ------------------def fred_series_monthly(series_id: str, start: str, end: str, how="mean") -> pd.Series:    if not FRED_KEY:        raise RuntimeError("Missing FRED_API_KEY")    url = FRED_BASE    params = {"series_id": series_id, "api_key": FRED_KEY, "file_type": "json",              "observation_start": start, "observation_end": end}    js = _get_json(url, params)    obs = js.get("observations", [])    if not obs:        return pd.Series(dtype=float)    df = pd.DataFrame(obs)    df["date"] = pd.to_datetime(df["date"])    df["value"] = pd.to_numeric(df["value"], errors="coerce")    s = df.set_index("date")["value"].astype(float)    # Use 'ME' (month-end) to avoid deprecation warning    return s.resample("ME").last() if how == "last" else s.resample("ME").mean()def get_macro_block(start: str, end: str) -> pd.DataFrame:    series = {        "FEDFUNDS": "fed_funds_rate",        "CPIAUCSL": "cpi_index",        "DGS10":    "us10y",        "UNRATE":   "unemployment_rate"    }    cols = {}    for sid, name in series.items():        cols[name] = fred_series_monthly(sid, start, end, how="mean")    macro = pd.DataFrame(cols).sort_index()    # Engineered features    if "cpi_index" in macro:        macro["inflation_yoy"] = macro["cpi_index"].pct_change(12) * 100    if "us10y" in macro:        macro["us10y_chg"] = macro["us10y"].diff(1)    if "fed_funds_rate" in macro:        macro["fedfunds_chg"] = macro["fed_funds_rate"].diff(1)    if "unemployment_rate" in macro:        macro["unrate_chg"] = macro["unemployment_rate"].diff(1)    diagnostics(macro, "macro monthly")    _to_csv(macro, f"{OUT_DIR}/macro_monthly.csv")    return macro# ------------------ Prices & sector / AI indexes (yfinance) ------------------def monthly_from_yf(tickers, start, end):    data = yf.download(tickers, start=start, end=end, progress=False, auto_adjust=True)    if isinstance(tickers, str) or (isinstance(tickers, list) and len(tickers) == 1):        close = data["Close"]        if isinstance(close, pd.Series):            close = close.to_frame(name=tickers if isinstance(tickers, str) else tickers[0])    else:        close = data["Close"]    m = close.resample("ME").last()    rets = m.pct_change()    return m, rets# ------------------ News sentiment (Polygon + Finnhub, optional) ------------------def simple_sentiment(text: str) -> float:    if not isinstance(text, str) or not text.strip():        return 0.0    if _VADER_OK:        sia = SentimentIntensityAnalyzer()        return float(sia.polarity_scores(text)["compound"])    # fallback keyword heuristic    t = text.lower()    pos = sum(w in t for w in ["beat","record","growth","surge","profit","upgrade","outperform","strong","rally"])    neg = sum(w in t for w in ["miss","cut","probe","lawsuit","downgrade","decline","headwind","weak","plunge"])    return (pos - neg) / 6.0def polygon_news_raw(ticker: str, api_key: str, start=None, end=None, max_pages=20, page_limit=1000, sleep=0.4):    cols = ["date","headline","summary","source"]    if not api_key:        return pd.DataFrame(columns=cols)    url = f"{POLY_BASE}/v2/reference/news"    params = {"ticker": ticker, "limit": min(page_limit, 1000), "order": "desc", "apiKey": api_key}    if start: params["published_utc.gte"] = pd.Timestamp(start).strftime("%Y-%m-%d")    if end:   params["published_utc.lte"] = pd.Timestamp(end).strftime("%Y-%m-%d")    all_rows, cursor, pages = [], None, 0    while True:        if cursor:            params["cursor"] = cursor        js = _get_json_soft(url, params)        if not js or "results" not in js or not js["results"]:            break        rows = js["results"]        for r in rows:            d  = pd.to_datetime(r.get("published_utc"), errors="coerce")            tl = r.get("title", "")            ds = r.get("description", "")            if pd.isna(d):                continue            if start and d < pd.Timestamp(start):                break            all_rows.append({"date": d, "headline": tl, "summary": ds, "source": "polygon"})        pages += 1        cursor = js.get("next_url") or js.get("next") or js.get("cursor")        if not cursor or pages >= max_pages:            break        time.sleep(sleep)    if not all_rows:        return pd.DataFrame(columns=cols)    df = pd.DataFrame(all_rows).sort_values("date")    return df[cols]def finnhub_news_raw(ticker: str, api_key: str, start: str, end: str, chunk="365D", sleep=0.3):    cols = ["date","headline","summary","source"]    if not api_key:        return pd.DataFrame(columns=cols)    start_ts = pd.Timestamp(start); end_ts = pd.Timestamp(end); step = pd.Timedelta(chunk)    frames, lo = [], start_ts    while lo <= end_ts:        hi = min(lo + step, end_ts)        url = f"{FINNHUB_BASE}/company-news"        params = {"symbol": ticker, "from": lo.date().isoformat(), "to": hi.date().isoformat(), "token": api_key}        js = _get_json_soft(url, params)        if isinstance(js, list) and js:            df = pd.DataFrame(js)            d  = pd.to_datetime(df.get("datetime"), unit="s", errors="coerce")            if d.isna().all():                d = pd.to_datetime(df.get("time"), unit="ms", errors="coerce")            df_out = pd.DataFrame({                "date": d,                "headline": df.get("headline", ""),                "summary":  df.get("summary", ""),                "source":   "finnhub"            }).dropna(subset=["date"])            if not df_out.empty:                frames.append(df_out)        lo = hi + pd.Timedelta("1D")        time.sleep(sleep)    if not frames:        return pd.DataFrame(columns=cols)    return pd.concat(frames, ignore_index=True).sort_values("date")[cols]def build_monthly_news_sentiment_combined(ticker: str, start: str, end: str, polygon_key: str, finnhub_key: str, dedup_within_days: int = 3) -> pd.DataFrame:    poly_df = polygon_news_raw(ticker, polygon_key, start=start, end=end)    fin_df  = finnhub_news_raw(ticker, finnhub_key, start, end)    combined = pd.concat([poly_df, fin_df], ignore_index=True)    if combined.empty:        cols = ["sent_mean","sent_count","sent_mean_weighted","sent_mean_poly","sent_count_poly","sent_mean_fin","sent_count_fin"]        return pd.DataFrame(columns=cols, dtype=float)    combined["date"] = pd.to_datetime(combined.get("date"), errors="coerce", utc=True)    combined = combined.dropna(subset=["date"]).copy()    try:        combined["date"] = combined["date"].dt.tz_convert(None)    except Exception:        pass    for col in ["headline","summary"]:        if col not in combined.columns:            combined[col] = ""        combined[col] = combined[col].fillna("").astype(str)    if dedup_within_days and dedup_within_days > 0:        rd = f"{dedup_within_days}D"        combined["date_round"] = combined["date"].dt.floor(rd)        combined = combined.drop_duplicates(subset=["date_round","headline"])    texts = combined["headline"] + ". " + combined["summary"]    combined["sent"] = texts.apply(simple_sentiment)    combined["month"] = combined["date"].dt.to_period("M").dt.to_timestamp("ME")    src = combined.groupby(["month","source"]).agg(sent_mean=("sent","mean"), sent_count=("sent","size")).reset_index()    src_piv = src.pivot(index="month", columns="source", values=["sent_mean","sent_count"])    # ensure expected columns exist    for c in [("sent_mean","polygon"), ("sent_mean","finnhub"), ("sent_count","polygon"), ("sent_count","finnhub")]:        if c not in src_piv.columns:            src_piv[c] = np.nan if "mean" in c[0] else 0    src_piv = src_piv.sort_index()    out = pd.DataFrame(index=src_piv.index)    out["sent_mean_poly"]  = src_piv[("sent_mean","polygon")]    out["sent_count_poly"] = src_piv[("sent_count","polygon")].fillna(0).astype(int)    out["sent_mean_fin"]   = src_piv[("sent_mean","finnhub")]    out["sent_count_fin"]  = src_piv[("sent_count","finnhub")].fillna(0).astype(int)    all_month = combined.groupby("month").agg(sent_mean=("sent","mean"), sent_count=("sent","size"))    out = out.join(all_month, how="outer")    num = (out["sent_mean_poly"].fillna(0) * out["sent_count_poly"].astype(float) +           out["sent_mean_fin"].fillna(0)  * out["sent_count_fin"].astype(float))    den = (out["sent_count_poly"].astype(float) + out["sent_count_fin"].astype(float))    out["sent_mean_weighted"] = np.where(den > 0, num / den, np.nan)    return out[["sent_mean","sent_count","sent_mean_weighted","sent_mean_poly","sent_count_poly","sent_mean_fin","sent_count_fin"]].sort_index()# ------------------ Earnings (Finnhub, optional) ------------------def finnhub_earnings(ticker: str, start: str, end: str) -> pd.DataFrame:    cols = ["date","epsActual","epsEstimate","surprisePercent"]    if not FINNHUB_KEY:        return pd.DataFrame(columns=cols)    url = f"{FINNHUB_BASE}/stock/earnings"    params = {"symbol": ticker, "token": FINNHUB_KEY}    js = _get_json_soft(url, params)    if not isinstance(js, list) or len(js) == 0:        url2 = f"{FINNHUB_BASE}/calendar/earnings"        params2 = {"from": start, "to": end, "token": FINNHUB_KEY}        js2 = _get_json_soft(url2, params2) or {}        df2 = pd.DataFrame(js2.get("earningsCalendar", []))        if df2.empty:            return pd.DataFrame(columns=cols)        df2["date"] = pd.to_datetime(df2["date"], errors="coerce")        df2 = df2[df2.get("symbol","") == ticker]        keep = [c for c in cols if c in df2.columns]        return df2[keep].dropna(subset=["date"]).sort_values("date")    df = pd.DataFrame(js)    if "date" in df.columns:        df["date"] = pd.to_datetime(df["date"], errors="coerce")    elif "period" in df.columns:        df["date"] = pd.to_datetime(df["period"], errors="coerce")    if "surprisePercent" not in df.columns and {"epsActual","epsEstimate"} <= set(df.columns):        with np.errstate(divide='ignore', invalid='ignore'):            df["surprisePercent"] = (df["epsActual"] - df["epsEstimate"]) / df["epsEstimate"] * 100.0    keep = [c for c in cols if c in df.columns]    return df[keep].dropna(subset=["date"]).sort_values("date")def monthly_earnings_features(ticker: str) -> pd.DataFrame:    df = finnhub_earnings(ticker, START, END)    if df.empty:        return pd.DataFrame(columns=["month","eps_surprise_mean","eps_surprise_last"]).set_index("month")    df["month"] = df["date"].dt.to_period("M").dt.to_timestamp("ME")    agg = df.groupby("month").agg(        eps_surprise_mean=("surprisePercent","mean"),        eps_surprise_last=("surprisePercent","last")    ).sort_index()    return agg# ------------------ Build per‑ticker enriched features ------------------def build_ticker_features(ticker: str, macro: pd.DataFrame, ixic_rets: pd.DataFrame, xlk_rets: pd.DataFrame, ai_ret_eqw: pd.DataFrame, news_sent_maps, earnings_maps) -> pd.DataFrame:    close, rets = monthly_from_yf(ticker, START, END)    t_ret = rets.rename(columns={rets.columns[0]: f"{ticker}_ret"})    feats = t_ret.join(ixic_rets, how="left").join(xlk_rets, how="left").join(ai_ret_eqw, how="left")    if ticker in news_sent_maps: feats = feats.join(news_sent_maps[ticker], how="left")    if ticker in earnings_maps:  feats = feats.join(earnings_maps[ticker],  how="left")    base_feats = ["inflation_yoy","us10y","us10y_chg","fed_funds_rate","fedfunds_chg","unemployment_rate","unrate_chg"]    macro_lagged = make_lags(macro, base_feats, lags=LAGS)    feats = feats.join(macro_lagged, how="left")    max_lag = max(LAGS) if LAGS else 0    if len(feats) > max_lag:        feats = feats.iloc[max_lag:]    return feats# ------------------ Main pipeline ------------------def main():    # Macro    macro = get_macro_block(START, END)    # Index & AI basket returns    ixic_close, ixic_rets = monthly_from_yf("^IXIC", START, END); ixic_rets.columns = ["ixic_ret"]    xlk_close,  xlk_rets  = monthly_from_yf("XLK",   START, END);  xlk_rets.columns  = ["xlk_ret"]    ai_close, ai_rets = monthly_from_yf(AI_BASKET, START, END)    ai_ret_eqw = ai_rets.mean(axis=1).to_frame(name="ai_basket_ret")    _to_csv(ixic_rets, f"{OUT_DIR}/ixic_rets.csv")    _to_csv(xlk_rets,  f"{OUT_DIR}/xlk_rets.csv")    _to_csv(ai_ret_eqw, f"{OUT_DIR}/ai_basket_rets.csv")    # News (optional) & Earnings (optional)    news_sent_maps, earnings_maps = {}, {}    if POLYGON_KEY or FINNHUB_KEY:        for t in TECH_TICKERS:            try:                s = build_monthly_news_sentiment_combined(t, START, END, POLYGON_KEY, FINNHUB_KEY)            except Exception as e:                print(f"[warn] news fail for {t}: {e}")                s = pd.DataFrame(columns=["sent_mean","sent_count","sent_mean_weighted","sent_mean_poly","sent_count_poly","sent_mean_fin","sent_count_fin"])            news_sent_maps[t] = s            _to_csv(s, f"{OUT_DIR}/{t}_news_sentiment_combined.csv")    else:        print("[info] No POLYGON/FINNHUB keys; skipping news sentiment.")    if FINNHUB_KEY:        for t in TECH_TICKERS:            try:                e = monthly_earnings_features(t)            except Exception as e2:                print(f"[warn] earnings fail for {t}: {e2}")                e = pd.DataFrame(columns=["eps_surprise_mean","eps_surprise_last"])            earnings_maps[t] = e            _to_csv(e, f"{OUT_DIR}/{t}_earnings_features.csv")    else:        print("[info] No FINNHUB key; skipping earnings features.")    # Build features per ticker + panel    all_feat = {}    for t in TECH_TICKERS:        print(f"Features for {t} ...")        ft = build_ticker_features(t, macro, ixic_rets, xlk_rets, ai_ret_eqw, news_sent_maps, earnings_maps)        diagnostics(ft, f"{t} features")        all_feat[t] = ft        _to_csv(ft, f"{OUT_DIR}/{t}_features_enriched.csv")    combined = pd.concat(all_feat, axis=1)    _to_csv(combined, f"{OUT_DIR}/tech_features_combined.csv")    # OLS (optional, only if statsmodels is available)    if _STATSMODELS_OK:        MIN_ROWS, MAX_FEATS = 12, 20        def fit_ols_safe(df: pd.DataFrame, target_col: str, min_rows: int = 12):            X = df.drop(columns=[target_col])            y = df[target_col]            aligned = pd.concat([y, X], axis=1).dropna()            info = {"rows_before": len(df), "rows_after_dropna": len(aligned), "n_features": X.shape[1]}            if len(aligned) < min_rows:                return None, info            y_clean = aligned.iloc[:,0]            X_clean = aligned.iloc[:,1:]            X_clean = sm.add_constant(X_clean, has_constant="add")            m = sm.OLS(y_clean, X_clean).fit()            return m, info        def top_features_by_coverage(df: pd.DataFrame, target_col: str, k: int = 20):            na_rates = df.drop(columns=[target_col]).isna().mean()            keep_feats = na_rates.sort_values().index[:k].tolist()            cols = [target_col] + keep_feats            return df[cols]        for t in TECH_TICKERS:            print(f"\n=== {t} OLS (enriched) ===")            df_t = all_feat[t].copy()            target = f"{t}_ret"            df_t_red = top_features_by_coverage(df_t, target, k=MAX_FEATS)            m, info = fit_ols_safe(df_t_red, target, min_rows=MIN_ROWS)            print(f"rows_before={info['rows_before']}, rows_after_dropna={info['rows_after_dropna']}, features={info['n_features']}")            if m is None:                print(f"Not enough data after dropna (need >= {MIN_ROWS}).")                continue            try:                print(m.summary())            except Exception as e:                print("Could not print full summary:", e)                print("Params:\\n", m.params)                print("R2:", getattr(m, "rsquared", None), "Adj R2:", getattr(m, "rsquared_adj", None))    else:        print("[info] statsmodels not installed; skipping OLS step. To enable: pip install statsmodels")if __name__ == "__main__":    main()

In [ ]:
main()  # run pipeline